In [17]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from pre_process import preprocess_graph_attributes
from datetime import datetime, timedelta
from pm4py.objects.conversion.log import converter as xes_converter
from pm4py.algo.filtering.log.attributes import attributes_filter
from pm4py.objects.log.obj import EventLog, Trace, Event
from utility import constants

In [18]:
logistic = pd.read_csv('logistics.csv')

In [19]:
#format timestamp
for idx, row in logistic.iterrows():
    logistic.loc[idx, 'time:timestamp'] = row['time:timestamp'].split('+')[0].split('.')[0]

In [20]:
#convert to log
logistic_log = xes_converter.apply(logistic, variant=xes_converter.Variants.TO_EVENT_LOG)

In [21]:
#rename columns
logistic = logistic.rename(columns = {'concept:name' : 'Equipment ID', 'time:timestamp' : 'Arrived Timestamp'})

In [22]:
#create empty columns 
columns_to_add = ['case:Init Datetime', 'case:Complete Datetime', 'EQTYP', 'Zone Name', 'Floor', 'case:Src Eqt', 'case:Dest Eqt', 'case:Src Loc', 'case:Dest Loc']

for i in columns_to_add:
    logistic[i] = ""

In [23]:
#add equipment atrributes
for idx, row in logistic.iterrows():
    
    logistic.loc[idx, 'EQTYP'] = row['Equipment ID'][0:2]
    logistic.loc[idx, 'Zone Name'] = row['Equipment ID'][3:4]
    logistic.loc[idx, 'Floor'] = row['Equipment ID'][2:3]
    logistic.loc[idx, 'case:Src Loc'] = 'CSS9J'
    logistic.loc[idx, 'case:Dest Loc'] = 'TPJB2'

In [24]:
#get case attributes
case_attributes = []

for case in logistic_log:
    attribute = [case[0]['time:timestamp'], case[len(case) - 1]['time:timestamp'], case[0]['concept:name'], case[len(case) - 1]['concept:name']]
    for i in range(len(case)):
        case_attributes.append(attribute)
    

In [25]:
#add case attributes
for idx, row in logistic.iterrows():
    logistic.loc[idx, 'case:Init Datetime'] = case_attributes[idx][0]
    logistic.loc[idx, 'case:Complete Datetime'] = case_attributes[idx][1]
    logistic.loc[idx, 'case:Src Eqt'] = case_attributes[idx][2]
    logistic.loc[idx, 'case:Dest Eqt'] = case_attributes[idx][3]

In [26]:
#convert to new log
new_log = xes_converter.apply(logistic, variant=xes_converter.Variants.TO_EVENT_LOG)

In [27]:
#add departed timestamp
departed_timestamp = []

for case in new_log:
    for idx, event in enumerate(case):
        if idx < len(case) - 1:
            departed_timestamp.append(case[idx+1]['Arrived Timestamp'])
        else:
            departed_timestamp.append(case.attributes['Complete Datetime'])

logistic['Departed Timestamp'] = departed_timestamp

In [28]:
#save as csv
logistic.to_csv('logistics_format.csv')